In [33]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from activ import load_data
from PyUoI.UoINMF import UoINMF
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from activ.data_normalization import data_normalization
from scipy.spatial.distance import pdist
from sklearn.model_selection import KFold
import pandas as pd
from activ.readfile import TrackTBIFile

In [34]:
fulldata = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/preprocessed_data_matrix.csv', encoding = "ISO-8859-1")
fulldata.shape

(586, 919)

In [37]:
subdata = load_data()
sub_oc_features = subdata.outcome_features
sub_bm_features = subdata.biomarker_features
oc_bases = nmfdata.outcome_bases
sub_oc = subdata.outcomes
sub_bm = subdata.biomarkers

In [36]:
sub_bm.shape

(247, 235)

In [38]:
sub = [106,108,110]

names = ['admGCSeyes', 'admGCSverb', 'admGCSmotor']

eyes = sub_bm[:,106]
verb = sub_bm[:,108]
motor = sub_bm[:,110]

count=0
patients = []

for i in range(247):
    score = eyes[i]+verb[i]+motor[i]
    if score >= 13 and score <= 15:
        patients.append(i)
        count+=1

bm_gcs = np.zeros((len(patients), sub_bm.shape[1]))
oc_gcs = np.zeros((len(patients), sub_oc.shape[1]))
for ii,p in enumerate(patients):
    bm_gcs[ii,:] = sub_bm[p,:]
    oc_gcs[ii,:] = sub_oc[p,:]
print(bm_gcs.shape)
print(oc_gcs.shape)

(210, 235)
(210, 177)


In [40]:
fname = '/Users/ahyeon/data/activ/gcs13-15/bm_gcs_13-15.h5'
TrackTBIFile.write(fname, bm_gcs, oc_gcs)

In [ ]:
raw = load_data()
labels = raw.outcome_features
nmfdata = load_data(uoinmf=True)
rawbm = raw.biomarkers #(247,235)
feature_names = raw.outcome_features
A = data_normalization.data_normalization(raw.outcomes, 'positive') #(247,177)
w = nmfdata.outcomes #(247,6)
h = nmfdata.outcome_bases #(6,177)

In [ ]:
def drop_x_percent(A,w,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    indices_lowest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_lowest_values])
    h_new = np.delete(h, indices_lowest_values, axis=1)
    A_new = np.delete(A, indices_lowest_values, axis=1) 
    return A_new, h_new, dropped_features

In [ ]:
percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_features_dropped,h_new, dropped_features = drop_x_percent(A,w,h,feature_names,p)
    fname = '/Users/ahyeon/data/activ/gcs13-15/bm_gcs_13-15.h5'.format(p)
    TrackTBIFile.write(fname, sub_bm, A_features_dropped)

In [23]:
help(TrackTBIFile)

Help on class TrackTBIFile in module activ.readfile:

class TrackTBIFile(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, filename)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  write(dest, biomarkers, outcomes, biomarker_features=None, outcome_features=None, patient_ids=None) from builtins.type
 |      Write biomarkers and outcomes to an HDF5 file
 |      
 |      Args:
 |          dest                : the path to an HDF5 file or the :class:`h5py.Group` to write to
 |          biomarkers          : the biomarker data with shape (n_samples, n_features)
 |          outcomes            : the outcomes data with shape (n_samples, n_features)
 |          biomarker_features  : the names of the biomarker features (optional)
 |          outcome_features    : the names of the outcome features (optional)
 |          patient_ids      